# Your favorite games AI - Trackmania AI - Training Pipeline

Travail réalisé par **Grégory Bassard** accompagné et supervisé par **Emmanuel De Salis** \
HE-Arc Ingénierie \
<img src="res\ARC_Ingenierie_RGB.jpg" width="250"/> \
*Tous droits réservés* ©


## 1. Initialisation du stème et logging
Configuration des outils de suivi (logs) et détection automatique du matériel d'accélération PyTorch.

In [ ]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('training.log')
        # logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [ ]:
import torch

device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)

logger.info(f'Using device: {device}')

## 2. Interface du jeu
Script d'automatisation pour lancer l'instance de *Trackmania Nations Forever* via TMInterface. Le code redimensionne et positionne les fenêtres.

In [ ]:
from script.TMInterface.TMInterface_setting import start_TMInterface, windows_resize, windows_move, get_screen_size

tm_path = r'C:\Program Files (x86)\TmNationsForever\TMInterface.exe'

screen_width, screen_height = get_screen_size()

width = int(screen_width / 2)
height = screen_height - 40

right_window_panel_x = screen_width - width + 8
right_window_panel_y = 0

tmInterface_window = start_TMInterface(tm_path)
windows_resize(tmInterface_window, width=width, height=height)

windows_move(tmInterface_window, x=right_window_panel_x, y=right_window_panel_y)

## 3. Définition de l'environnement (RL environment)
Importation de l'environnement personnalisé.

In [ ]:
# from custom_env.TMEnv_blind import TrackmaniaEnv
from custom_env.TMEnv_road import TrackmaniaEnv

## 4. Architecture de l'agent (modèle DQN)
Instanciation de l'agent utilisant un réseau Deep Q-Network.

In [ ]:
# from model.DQN_model_blind import Agent
from model.DQN_model_road import Agent

## 5. Données de position de la route sur le circuit
Chargement des coordonnées de la route

In [ ]:
map_name = "challenge_2"

road_pos_array = open(f'data/road_positions/{map_name}.txt', 'r').readlines()
road_pos_array = [list(map(float, line.strip().split(','))) for line in road_pos_array]

print(f"Loaded {len(road_pos_array)} road positions for map {map_name}\n{road_pos_array[:10]}")

## 6. Training loop
Hyperparamètres et lancement de l'interaction entre l'Agent et l'Environnement pour démarrer l'apprentissage.

In [ ]:
print("-----------MAIN-----------")
SERVER_NAME = 'TMInterface0' # nom du serveur TMInterface utilisé (doit correspondre à celui dans TMInterface.exe, par défaut 'TMInterface0')

speed_IG = 3 # multiplicateur de la vitesse en jeu (IG = in game)
num_episodes = 5000 # nombre de courses d'entrainement

hidden_size = 512 # taille des couches cachées du réseau de neurones

max_race_time = 40.0  # temps maximum d'une course en secondes

# initialisation de l'environnement
env = TrackmaniaEnv(
        speed=speed_IG,
        server_name=SERVER_NAME,
        map_name=map_name,
        max_race_time=max_race_time,
        road_pos=road_pos_array,
        off_track_threshold_y=40.0
    )

# initialisation de l'agent
agent = Agent(env, device, hidden_size, speed_IG, None, tmInterface_window)

# entrainement de l'agent
agent.train(num_episodes=num_episodes, plot=True)

print("-----------END-----------")

## 7. Sauvegarde et Fermeture
Exportation du modèle entraîné dans un fichier *.pth* dans le dossier *./output* avec comme nom de fichier le nom de la map, la vitesse IG, la version (incrémenté automatiquement) et 'road' pour indiquer qu'il s'agit du modèle utilisant la position du centre de la route. Et fermeture de la connexion avec le jeu

In [ ]:
try:
    agent.save(output_folder='./output', map_name=map_name, additional_info='road')
except Exception as e:
    env.logger.error(f"Error saving agent: {e}")
    print(f"Error saving agent: {e}")

env.close()